In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

data_train = pd.read_csv("/content/gdrive/My Drive/DoDef/task3_training (2).tsv" ,delimiter = "\t", encoding="utf-8-sig").fillna(method="ffill")
data_test = pd.read_csv("/content/gdrive/My Drive/DoDef/task3_validation (1).tsv", delimiter = "\t", encoding="utf-8-sig").fillna(method="ffill")

In [0]:
data = pd.concat([data_train,data_test])

In [0]:
data= data.loc[:, ~data.columns.str.contains('^Unnamed')]

In [0]:
data.head()

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,331187619096588288,NaN,NaN,NaN,NaN,ofloxacin,@seefisch:oral drugs for pyelonephritis:ciprof...,NaN,NaN
1,332227554956161024,NaN,NaN,NaN,NaN,trazodone,happy for wellbutrin; has similar effects as a...,NaN,NaN
2,332448217490944000,NaN,NaN,NaN,NaN,lamotrigine,@stilgarg i'm ok ty have an official diagnosis...,NaN,NaN
3,332977955754110976,NaN,NaN,NaN,NaN,cymbalta,i'm soo depressed cymbalta couldn't help me .,NaN,NaN
4,333674203331051520,NaN,NaN,NaN,NaN,seroquel,time for my daily afternoon relaxation ritual ...,NaN,NaN


In [0]:
data = data.dropna(subset=['type'])

In [0]:
data.head()

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
27,342314998904786945,42.0,53.0,ADR,lost vision,cymbalta,"#cymbalta withdrawal has reached a peak, lost ...",10047522.0,vision loss
28,342314998904786945,11.0,21.0,ADR,withdrawal,cymbalta,"#cymbalta withdrawal has reached a peak, lost ...",10048010.0,withdrawal syndrome
29,342322703556038657,27.0,35.0,ADR,nauseous,cipro,i hate cipro! #antibiotic #nauseous #cf #hospi...,10028823.0,nauseous
30,342349802601844737,109.0,118.0,ADR,can't cum,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...,10021574.0,inability to orgasm
31,342349802601844737,101.0,104.0,ADR,fat,seroquel,@luckystubbs reppin zoloft&amp;seroquel since ...,10047896.0,weight gain


In [0]:
actual_terms = []
extracted_mentions = []
for index,row in data.iterrows():
  actual_terms.append(row.meddra_term)
  extracted_mentions.append(row.extraction)

In [0]:
import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)
def embed(input):
  return model(input)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
extracted_embeddings = embed(extracted_mentions)

In [0]:
actual_embeddings = embed(np.unique(actual_terms))

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
actual_labels = list(le.fit_transform(actual_terms))

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cosine = cosine_similarity(extracted_embeddings, actual_embeddings)
indexes = list(np.argmax(cosine, axis=1))

In [0]:
unique_actual = np.unique(actual_terms)
predicted = []
for item in indexes:
  predicted.append(unique_actual[item])

In [0]:
from sklearn.metrics import classification_report

print (classification_report(predicted, actual_terms))

                                  precision    recall  f1-score   support

                             ADD       1.00      0.25      0.40         4
                MS-like syndrome       0.00      0.00      0.00         0
                       SLE flare       1.00      0.75      0.86         4
                   Schizophrenia       1.00      0.25      0.40         4
                    TMJ syndrome       1.00      1.00      1.00         1
                  abdominal pain       1.00      1.00      1.00         1
                 abnormal dreams       0.00      0.00      0.00         0
                            ache       0.92      0.52      0.67        21
      achilles tendon discomfort       1.00      1.00      1.00         1
                 aching in knees       0.50      0.17      0.25         6
                  aching in limb       1.00      0.50      0.67         2
                            acne       1.00      1.00      1.00         5
                       addiction     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
print (predicted[2],actual_terms[2])

nauseous nauseous
